In [ ]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
apiKey = os.environ["open_ai_key"]


In [ ]:
def load_csvs():
    folder_path = "../202401_NFs"
    dataframes = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            df_name = os.path.splitext(filename)[0]
            try:
                df = pd.read_csv(os.path.join(folder_path, filename))
                dataframes[df_name] = df
                print(f"Carregado: {filename} com {len(df)} linhas.")
            except Exception as e:
                print(f"Erro ao carregar {filename}: {e}")
    return dataframes

dataframes = load_csvs()

Carregado: 202401_NFs_Cabecalho.csv com 100 linhas.
Carregado: 202401_NFs_Itens.csv com 565 linhas.


In [8]:
def generate_context(dataframes):
    context = ""
    for name, df in dataframes.items():
        context += f"Dataset: {name}\n"
        context += f"Colunas: {', '.join(df.columns)}\n"
        context += f"Total de linhas: {len(df)}\n\n"
    return context

In [9]:
template = """
Você é um assistente Python que ajuda a responder perguntas sobre múltiplos datasets carregados como DataFrames Pandas.

Aqui está um resumo dos datasets disponíveis:

{dataset_context}

Pergunta do usuário:
{user_question}
"""

prompt_template = PromptTemplate.from_template(template)

In [18]:
def ask_question(user_question):
    context = generate_context(dataframes)
    dataset_names = ", ".join(dataframes.keys())

    return prompt_template.format(
        dataset_context=context,
        user_question=user_question
    )

In [16]:
llm = ChatOpenAI(model="gpt-4o",api_key=apiKey)

In [20]:
pergunta = ask_question("quais colunas permitem relação entre os arquivos?")

response = llm.invoke(pergunta).content

print(response)

Para identificar quais colunas permitem a relação entre os dois datasets, precisamos procurar colunas que estejam presentes em ambos os datasets e que possam ser utilizadas para correlacionar as informações. Com base no resumo fornecido, as seguintes colunas permitem a relação entre os dois datasets:

1. **CHAVE DE ACESSO**: Esta é uma coluna que serve como identificador único para as notas fiscais e está presente em ambos os datasets. Ela pode ser usada para associar diretamente as informações de cabeçalho com os itens correspondentes.

2. **MODELO**: Embora essa coluna esteja presente em ambos os datasets, ela não é exclusiva para identificar uma relação direta, mas pode ser utilizada como critério adicional.

3. **SÉRIE**: Similar ao "MODELO", ela está presente em ambos os datasets e pode ajudar como um critério adicional.

4. **NÚMERO**: Esse campo, juntamente com o "MODELO" e "SÉRIE", pode ajudar a identificar um documento fiscal de forma única, especialmente se a "CHAVE DE ACESSO